In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

/home/ahmad/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Supaya hasil acaknya sama terus
np.random.seed(42)

# Bikin 1000 Data Pelanggan
n_samples = 1000

data = {
    'Tenure': np.random.randint(1, 72, n_samples), # 1 s/d 72 bulan
    'MonthlyCharges': np.random.randint(20, 100, n_samples), # 20 s/d 100 dollar
    'PaperlessBilling': np.random.randint(0, 2, n_samples),
    'SeniorCitizen': np.random.randint(0, 2, n_samples),
    'Partner': np.random.randint(0, 2, n_samples),
    'Dependents': np.random.randint(0, 2, n_samples),
    'PhoneService': np.random.randint(0, 2, n_samples),
    'LuckyNumber': np.random.randint(1, 1000, n_samples) # Fitur Sampah (Random)
}

df = pd.DataFrame(data)

# Bikin kolom TotalCharges (Tenure * Monthly) + sedikit acak
df['TotalCharges'] = df['Tenure'] * df['MonthlyCharges']

# Bikin Target CHURN (Logika: Kalau Tenure kecil & Tagihan Mahal -> Cenderung Churn)
# Ini rumus rahasia biar ada polanya
score = (100 - df['Tenure']) + (df['MonthlyCharges'] * 0.5) + np.random.normal(0, 20, n_samples)
df['Churn'] = (score > 80).astype(int) # 1 = Churn, 0 = Setia
df

,Tenure,MonthlyCharges,PaperlessBilling,SeniorCitizen,Partner,Dependents,PhoneService,LuckyNumber,TotalCharges,Churn
0,52,65,0,1,0,1,1,963,3380,0
1,15,35,0,0,0,0,0,193,525,1
2,61,87,0,0,0,0,0,895,5307,0
3,21,56,1,1,1,0,1,58,1176,1
4,24,73,0,1,1,0,1,437,1752,1
...,...,...,...,...,...,...,...,...,...,...
995,51,99,1,1,0,0,1,352,5049,1
996,8,26,0,0,0,1,1,701,208,1
997,34,71,1,1,1,1,0,928,2414,1
998,35,33,0,0,1,0,1,256,1155,1


In [10]:
x = df.drop(columns='Churn')
y = df['Churn']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
selector_filter = SelectKBest(score_func=chi2, k=3)

#fit dan transform 
x_train_filter = selector_filter.fit_transform(x_train, y_train)
x_test_filter = selector_filter.transform(x_test)

cols_filter = selector_filter.get_support(indices=True)
x.columns[cols_filter]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [50]:
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model_lr = LogisticRegression(solver='liblinear')

#siapkan rfe
selector_rfe = RFE(estimator=model_lr, n_features_to_select=3)

#fit dan transform
x_train_rfe = selector_rfe.fit_transform(x_train_scaled, y_train)
x_test_rfe = selector_rfe.transform(x_test_scaled)

cols_rfe = selector_rfe.get_support(indices=True)
x.columns[cols_rfe]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [43]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(x_train, y_train)

#ambil skor kepentingan
importance = model_rf.feature_importances_
indices_rf = np.argsort(importance)[::-1][:3]
indices_rf

array([0, 8, 1])

In [48]:
x_train_emb = x_train.iloc[:, indices_rf]
x_test_emb = x_test.iloc[:, indices_rf]
x_train_emb

,Tenure,TotalCharges,MonthlyCharges
29,64,3584,56
535,15,465,31
695,7,518,74
557,17,918,54
836,6,270,45
...,...,...,...
106,15,885,59
270,46,1288,28
860,47,3102,66
435,37,2664,72


# SKENARIO 2

In [52]:
import pandas as pd
import numpy as np

np.random.seed(42)
n = 1000

# Membuat Data Campuran
data = {
    'Satisfaction': np.random.rand(n), # Angka 0 sampai 1
    'LastEval': np.random.rand(n),     # Angka 0 sampai 1
    'Projects': np.random.randint(2, 8, n), # 2 sampai 7 proyek
    'Salary': np.random.choice(['Low', 'Medium', 'High'], n), # TEKS
    'Dept': np.random.choice(['Sales', 'IT', 'HR'], n)        # TEKS
}

df_hr = pd.DataFrame(data)

# Logika Target (Resign):
# Orang Resign biasanya: Gaji Rendah (Low) DAN Kerja Banyak (Projects > 5) DAN Kepuasan Rendah
# Kita buat logika matematikanya:
conditions = (
    (df_hr['Salary'] == 'Low') & 
    (df_hr['Projects'] > 5) & 
    (df_hr['Satisfaction'] < 0.5)
)
# Jika kondisi terpenuhi, peluang resign tinggi (80%), jika tidak kecil (10%)
df_hr['Left'] = np.where(conditions, 
                      np.random.choice([1, 0], n, p=[0.9, 0.1]), 
                      np.random.choice([1, 0], n, p=[0.1, 0.9]))

print("Data Mentah (Masih ada Teks):")
display(df_hr.head())

Data Mentah (Masih ada Teks):


,Satisfaction,LastEval,Projects,Salary,Dept,Left
0,0.374540,0.185133,5,Medium,Sales,0
1,0.950714,0.541901,4,Low,HR,0
2,0.731994,0.872946,5,Low,Sales,0
3,0.598658,0.732225,7,High,Sales,0
4,0.156019,0.806561,3,Low,IT,0


In [53]:
df_hr

,Satisfaction,LastEval,Projects,Salary,Dept,Left
0,0.374540,0.185133,5,Medium,Sales,0
1,0.950714,0.541901,4,Low,HR,0
2,0.731994,0.872946,5,Low,Sales,0
3,0.598658,0.732225,7,High,Sales,0
4,0.156019,0.806561,3,Low,IT,0
...,...,...,...,...,...,...
995,0.091582,0.656955,2,Medium,HR,0
996,0.917314,0.956615,2,Low,HR,0
997,0.136819,0.068958,7,Low,HR,1
998,0.950237,0.057055,4,High,IT,0


In [87]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
le = LabelEncoder()

df_encoded['Salary'] = le.fit_transform(df['Salary'])
df_encoded['Dept'] = le.fit_transform(df['Dept'])

x = df_encoded.drop(columns='Left')
y = df_encoded['Left']

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


selector = SelectKBest(score_func=chi2, k=3)

x_new = selector.fit_transform(x, y)
selected_cols = selector.get_support(indices=True)

x.columns[selected_cols]

Index(['Satisfaction', 'Projects', 'Salary'], dtype='object')

In [105]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

model_lr = LogisticRegression(solver='liblinear')
rfe = RFE(estimator=model_lr, n_features_to_select=3)

rfe.fit(x_scaled, y)
rfe_cols = rfe.get_support(indices=True)
x.columns[rfe_cols]

Index(['Satisfaction', 'LastEval', 'Projects'], dtype='object')

In [113]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x, y)


importances = rf.feature_importances_
top3_indices = np.argsort(importances[::-1][:3])
x.columns[top3_indices]

Index(['Satisfaction', 'Projects', 'LastEval'], dtype='object')